In [1]:
import os
import pickle
import numpy as np
import tcbench as tcb
import matplotlib.pyplot as plt

from scipy.spatial.distance import cdist, pdist, squareform
from sklearn.cluster import KMeans, DBSCAN
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score, adjusted_rand_score, normalized_mutual_info_score
from sklearn.preprocessing import StandardScaler

In [5]:
vectors_baseline = np.load('artifacts-mirage19/baseline_vectors.npy')
labels_baseline = np.load('artifacts-mirage19/baseline_labels.npy')
vectors_embeddings = np.load('artifacts-mirage19/embeddings_vectors.npy')
labels_embeddings = np.load('artifacts-mirage19/embeddings_labels.npy')

In [6]:
def kmeans (feature_matrix, k):
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans_clusters = kmeans.fit_predict(feature_matrix)
    return kmeans_clusters

kmeans_baseline = kmeans(vectors_baseline, 20)
kmeans_embeddings = kmeans(vectors_embeddings, 20)

In [7]:
def dbscan (feature_matrix):
    dbscan = DBSCAN(eps = 0.2, min_samples = 100)
    dbscan_clusters = dbscan.fit_predict(feature_matrix)
    return dbscan_clusters

dbscan_baseline = dbscan(vectors_baseline)
dbscan_embeddings = dbscan(vectors_embeddings)

In [8]:
sil_score_baseline = silhouette_score(vectors_baseline, kmeans_baseline)
print(f"Baseline Silhouette Score: {sil_score_baseline:.4f}")
sil_score_embeddings = silhouette_score(vectors_embeddings, kmeans_embeddings)
print(f"Embeddings Silhouette Score: {sil_score_embeddings:.4f}")

ari_score_baseline = adjusted_rand_score(labels_baseline, kmeans_baseline)
print(f"Baseline Adjusted Rand Index (ARI): {ari_score_baseline:.4f}")
ari_score_embeddings = adjusted_rand_score(labels_embeddings, kmeans_embeddings)
print(f"Embeddings Adjusted Rand Index (ARI): {ari_score_embeddings:.4f}")

nmi_score_baseline = normalized_mutual_info_score(labels_baseline, kmeans_baseline)
print(f"Baseline Normalized Mutual Info (NMI): {nmi_score_baseline:.4f}")
nmi_score_embeddings = normalized_mutual_info_score(labels_embeddings, kmeans_embeddings)
print(f"Embeddings Normalized Mutual Info (NMI): {nmi_score_embeddings:.4f}")

Baseline Silhouette Score: 0.3360
Embeddings Silhouette Score: 0.1194
Baseline Adjusted Rand Index (ARI): 0.0528
Embeddings Adjusted Rand Index (ARI): 0.3246
Baseline Normalized Mutual Info (NMI): 0.1219
Embeddings Normalized Mutual Info (NMI): 0.4681


In [9]:
valid_points_baseline = dbscan_baseline != -1
valid_points_embeddings = dbscan_embeddings != -1
sil_score_baseline = silhouette_score(vectors_baseline[valid_points_baseline], dbscan_baseline[valid_points_baseline])
print(f"Baseline Silhouette Score: {sil_score_baseline:.4f}")
sil_score_embeddings = silhouette_score(vectors_embeddings[valid_points_embeddings], dbscan_embeddings[valid_points_embeddings])
print(f"Embeddings Silhouette Score: {sil_score_embeddings:.4f}")

ari_score_baseline = adjusted_rand_score(labels_baseline, dbscan_baseline)
print(f"Baseline Adjusted Rand Index (ARI): {ari_score_baseline:.4f}")
ari_score_embeddings = adjusted_rand_score(labels_embeddings, dbscan_embeddings)
print(f"Embeddings Adjusted Rand Index (ARI): {ari_score_embeddings:.4f}")

nmi_score_baseline = normalized_mutual_info_score(labels_baseline, dbscan_baseline)
print(f"Baseline Normalized Mutual Info (NMI): {nmi_score_baseline:.4f}")
nmi_score_embeddings = normalized_mutual_info_score(labels_embeddings, dbscan_embeddings)
print(f"Embeddings Normalized Mutual Info (NMI): {nmi_score_embeddings:.4f}")

Baseline Silhouette Score: -0.0252
Embeddings Silhouette Score: 0.9399
Baseline Adjusted Rand Index (ARI): 0.0141
Embeddings Adjusted Rand Index (ARI): -0.0020
Baseline Normalized Mutual Info (NMI): 0.1131
Embeddings Normalized Mutual Info (NMI): 0.0954


In [6]:
def calculate_within_class_distances(feature_matrix, true_labels, metric):
    class_distances = {}
    unique_labels = np.unique(true_labels)

    for label in unique_labels:
        class_indices = np.where(true_labels == label)[0]
        class_features = feature_matrix[class_indices]
        distances = squareform(pdist(class_features, metric))
        class_distances[label] = distances

    return class_distances

def calculate_between_class_distances(feature_matrix, true_labels, metric):
    class_distances = {}
    unique_labels = np.unique(true_labels)

    for label in unique_labels:
        class_indices = np.where(true_labels == label)[0]
        other_indices = np.where(true_labels != label)[0]
        class_features = feature_matrix[class_indices]
        other_features = feature_matrix[other_indices]
        distances = cdist(class_features, other_features, metric)
        class_distances[label] = distances

    return class_distances

In [11]:
labels_baseline[:2]

array([0, 0])

In [12]:
test_distances_within_baseline = calculate_within_class_distances(vectors_baseline[:2], labels_baseline[:2], metric='euclidean')
test_distances_within_baseline

{0: array([[0.        , 1.01932718],
        [1.01932718, 0.        ]])}

In [13]:
labels_embeddings[:2]

array([11, 18])

In [14]:
test_distances_between_embeddings = calculate_between_class_distances(vectors_embeddings[:2], labels_embeddings[:2], metric='euclidean')
test_distances_between_embeddings

{11: array([[3.61261807]]), 18: array([[3.61261807]])}

In [15]:
test_distances_between_embeddings_cosine = calculate_between_class_distances(vectors_embeddings[:2], labels_embeddings[:2], metric='cosine')
test_distances_between_embeddings_cosine

{11: array([[0.2227783]]), 18: array([[0.2227783]])}

In [16]:
test_similarities_between_embeddings_cosine = {
        label: 1 - distances for label, distances in test_distances_between_embeddings_cosine.items()
    }
test_similarities_between_embeddings_cosine

{11: array([[0.7772217]]), 18: array([[0.7772217]])}

In [ ]:
within_class_file = "/home/ev357/rds/hpc-work/baseline_within.pkl"
between_class_file = "/home/ev357/rds/hpc-work/baseline_between.pkl"

if os.path.exists(within_class_file) and os.path.exists(between_class_file):
    with open(within_class_file, 'rb') as f:
        distances_within_baseline = pickle.load(f)
    with open(between_class_file, 'rb') as f:
        distances_between_baseline = pickle.load(f)
else:
    distances_within_baseline = calculate_within_class_distances(vectors_baseline, labels_baseline, metric='euclidean')
    distances_between_baseline = calculate_between_class_distances(vectors_baseline, labels_baseline, metric='euclidean')
    
    with open(within_class_file, 'wb') as f:
        pickle.dump(distances_within_baseline, f)
    with open(between_class_file, 'wb') as f:
        pickle.dump(distances_between_baseline, f)

In [ ]:
within_class_file = "/home/ev357/rds/hpc-work/embeddings_within.npy"
between_class_file = "/home/ev357/rds/hpc-work/embeddings_between.npy"

if os.path.exists(within_class_file) and os.path.exists(between_class_file):
    with open(within_class_file, 'rb') as f:
        distances_within_embeddings = pickle.load(f)
    with open(between_class_file, 'rb') as f:
        distances_between_embeddings = pickle.load(f)
else:
    distances_within_embeddings = calculate_within_class_distances(vectors_embeddings, labels_embeddings, metric='euclidean')
    distances_between_embeddings = calculate_between_class_distances(vectors_embeddings, labels_embeddings, metric='euclidean')
    
    with open(within_class_file, 'wb') as f:
        pickle.dump(distances_within_embeddings, f)
    with open(between_class_file, 'wb') as f:
        pickle.dump(distances_between_embeddings, f)

In [ ]:
within_class_file = "/home/ev357/rds/hpc-work/embeddings_cosine_within.npy"
between_class_file = "/home/ev357/rds/hpc-work/embeddings_cosine_between.npy"

if os.path.exists(within_class_file) and os.path.exists(between_class_file):
    with open(within_class_file, 'rb') as f:
        distances_within_embeddings_cosine = pickle.load(f)
    with open(between_class_file, 'rb') as f:
        distances_between_embeddings_cosine = pickle.load(f)
else:
    distances_within_embeddings_cosine = calculate_within_class_distances(vectors_embeddings, labels_embeddings, metric='cosine')
    distances_between_embeddings_cosine = calculate_between_class_distances(vectors_embeddings, labels_embeddings, metric='cosine')
    
    with open(within_class_file, 'wb') as f:
        pickle.dump(distances_within_embeddings_cosine, f)
    with open(between_class_file, 'wb') as f:
        pickle.dump(distances_between_embeddings_cosine, f)

In [ ]:
def plot_distance_histograms(class_distances_1, class_distances_2):
    fig, axes = plt.subplots(4, 5, figsize=(20, 15))
    axes = axes.flatten()

    for i, ((label1, distances1), (label2, distances2)) in enumerate(zip(class_distances_1.items(), class_distances_2.items())):
        ax = axes[i]
        distances1 = distances1[np.triu_indices_from(distances1, k=1)]
        distances2 = distances2.flatten()
        
        ax.hist(distances1, bins=30, alpha=0.5, color='blue', label='Within class', density=True)
        ax.hist(distances2, bins=30, alpha=0.5, color='red', label='Between classes', density=True)
        
        ax.set_title(f'Class {label1}')
        ax.set_xlabel('Distance')
        ax.set_ylabel('Density')
        ax.legend()

    plt.tight_layout()
    plt.show()

In [ ]:
plot_distance_histograms(distances_within_baseline, distances_between_baseline)

In [ ]:
plot_distance_histograms(distances_within_embeddings, distances_between_embeddings)

In [ ]:
plot_distance_histograms(distances_within_embeddings_cosine, distances_between_embeddings_cosine)

In [ ]:
within_class_file = "/home/ev357/rds/hpc-work/embeddings_cosine_similarity_within.npy"
between_class_file = "/home/ev357/rds/hpc-work/embeddings_cosine_similarity_between.npy"

if os.path.exists(within_class_file) and os.path.exists(between_class_file):
    with open(within_class_file, 'rb') as f:
        similarity_within_embeddings_cosine = pickle.load(f)
    with open(between_class_file, 'rb') as f:
        similarity_between_embeddings_cosine = pickle.load(f)
else:
    similarity_within_embeddings_cosine = {
        label: 1 - distances for label, distances in distances_within_embeddings_cosine.items()
    }
    similarity_between_embeddings_cosine = {
        label: 1 - distances for label, distances in distances_between_embeddings_cosine.items()
    }
    
    with open(within_class_file, 'wb') as f:
        pickle.dump(similarity_within_embeddings_cosine, f)
    with open(between_class_file, 'wb') as f:
        pickle.dump(similarity_between_embeddings_cosine, f)

In [ ]:
plot_distance_histograms(similarity_within_embeddings_cosine, similarity_between_embeddings_cosine)

In [ ]:
def find_min_within(within_class_distances):
    min_distances = {}
    for label, distances in within_class_distances.items():
        np.fill_diagonal(distances, np.inf)
        min_distances[label] = np.min(distances, axis=1)
    return min_distances

def find_min_between(between_class_distances):
    min_distances = {}
    for label, distances in between_class_distances.items():
        min_distances[label] = np.min(distances, axis=1)
    return min_distances

def find_max_within(within_class_distances):
    max_distances = {}
    for label, distances in within_class_distances.items():
        distances = np.where(np.isinf(distances), np.nan, distances)
        np.fill_diagonal(distances, np.nan)
        max_distances[label] = np.nanmax(distances, axis=1)
    return max_distances

def find_max_between(between_class_distances):
    max_distances = {}
    for label, distances in between_class_distances.items():
        distances = np.where(np.isinf(distances), np.nan, distances)
        max_distances[label] = np.nanmax(distances, axis=1)
    return max_distances

In [ ]:
min_within_baseline = find_min_within(distances_within_baseline)
min_between_baseline = find_min_between(distances_between_baseline)
min_within_embeddings = find_min_within(distances_within_embeddings)
min_between_embeddings = find_min_between(distances_between_embeddings)
min_within_embeddings_cosine = find_min_within(distances_within_embeddings_cosine)
min_between_embeddings_cosine = find_min_between(distances_between_embeddings_cosine)
max_within_embeddings_cosine_similarity = find_max_within(similarity_within_embeddings_cosine)
max_between_embeddings_cosine_similarity = find_max_between(similarity_between_embeddings_cosine)

In [ ]:
def plot_min_histograms(min_within, min_between, xlim=1):
    fig, axes = plt.subplots(4, 5, figsize=(20, 15))
    axes = axes.flatten()

    for i, (label, min_within_distances) in enumerate(min_within.items()):
        ax = axes[i]
        min_between_distances = min_between[label]
        
        ax.hist(min_within_distances, bins=30, alpha=0.5, color='blue', label='Min within', density=True)
        ax.hist(min_between_distances, bins=30, alpha=0.5, color='red', label='Min between', density=True)
        
        ax.set_title(f'Class {label}')
        ax.set_xlabel('Distance')
        ax.set_ylabel('Density')
        ax.set_xlim(0, xlim)
        ax.legend()

    plt.tight_layout()
    plt.show()


def plot_max_histograms(max_within, max_between, xlim=0):
    fig, axes = plt.subplots(4, 5, figsize=(20, 15))
    axes = axes.flatten()

    for i, (label, max_within_distances) in enumerate(max_within.items()):
        ax = axes[i]
        max_between_distances = max_between[label]
        ax.hist(max_within_distances, bins=30, alpha=0.5, color='blue', label='Max within', density=True)
        ax.hist(max_between_distances, bins=30, alpha=0.5, color='red', label='Max between', density=True)
        
        ax.set_title(f'Class {label}')
        ax.set_xlabel('Distance')
        ax.set_ylabel('Density')
        ax.set_xlim(xlim, 1)
        ax.legend()

    plt.tight_layout()
    plt.show()

In [1]:
plot_min_histograms(min_within_baseline, min_between_baseline)

NameError: name 'plot_min_histograms' is not defined